# Tarefa 1 

 Os dados da PNADc referentes ao quarto trimestre de 2021 estarão disponíveis em:
 
  [IBGE - Pesquisa Nacional por Amostra de Domicílios Contínua Trimestral](https://www.ibge.gov.br/estatisticas/sociais/trabalho/9173-pesquisa-nacional-por-amostra-de-domicilios-continua-trimestral.html?edicao=20653&t=downloads)

In [35]:
# Importando pacotes que irei utilizar

import pandas as pd
import numpy as np
from pathlib import Path

Definindo caminhos relativos no caminho do dicionário -- 4°tri de 2021 que está em formato xls

In [36]:
# Define o caminho base
base_path = Path.cwd()

# Define o caminho relativo para a pasta de dados
data_folder = base_path / "Dados"

# Define o caminho relativo para o arquivo de dicionário
dictionary_path = data_folder / "Dicionario.xls"

# Lê o arquivo de dicionário básico PNAD contínua
dicionario = pd.read_excel(dictionary_path, skiprows=1)


# Tarefa 02

Os arquivos FWF possuem um formato de leitura pouco intuitivo, tornando praticamente inviável a extração de informações a partir da simples visualização dos mesmos. O Pandas não tem a capacidade de determinar o tamanho das colunas automaticamente, resultando na geração de um DataFrame confuso. Para solucionar essa questão, podemos utilizar o parâmetro "widths", que requer uma lista de números inteiros indicando o tamanho esperado de cada coluna. Faremos isso aliado a função read_fwl do PANDAS para importar os dados em um formato amigável. 

In [37]:
# Selecionando a variável tamanho do dicionário e excluindo os NA, depois transformando em inteiros, em seguida transformando numa lista.
tamanho_sem_na = dicionario['Tamanho'].dropna().astype(int).tolist()

## Definindo caminho relativo de dados da PNAD
pnad_path = data_folder / "PNADC_042021.txt"

## Importando os dados da PNAD.
pnad2021 = pd.read_fwf(pnad_path, header = None, widths=tamanho_sem_na, 
                       dtype=str, float_format="%.2f")

## pegando o código da variável no dicionário e excluindo os Na's
codigo = dicionario['Código\nda\nvariável'].dropna()     

## Colocando o código da variável como nome da coluna
pnad2021 = pnad2021.rename(columns=dict(zip(pnad2021.columns, codigo)))

# Printando a pnad2021
print(pnad2021)


         Ano Trimestre  UF Capital RM_RIDE        UPA  Estrato V1008 V1014  \
0       2021         4  11      11     NaN  110000016  1110011    01    09   
1       2021         4  11      11     NaN  110000016  1110011    01    09   
2       2021         4  11      11     NaN  110000016  1110011    01    09   
3       2021         4  11      11     NaN  110000016  1110011    01    09   
4       2021         4  11      11     NaN  110000016  1110011    01    09   
...      ...       ...  ..     ...     ...        ...      ...   ...   ...   
461790  2021         4  53      53     NaN  530059517  5310211    11    09   
461791  2021         4  53      53     NaN  530059517  5310211    14    09   
461792  2021         4  53      53     NaN  530059517  5310211    14    09   
461793  2021         4  53      53     NaN  530059517  5310211    14    09   
461794  2021         4  53      53     NaN  530059517  5310211    14    09   

       V1016  ...         V1028191         V1028192         V10

Agora, vou realizar uma estimativa amostral do número de pessoas em cada unidade federativa, levando em consideração os pesos amostrais. A variável V1028 é o peso amostral na PNADc. O resultado final abaixo é a estimativa amostral da população por unidade federativa. 

In [38]:
## Covertendno 'V1028' em float. V1028 é o peso 
pnad2021['V1028'] = pnad2021['V1028'].astype(float)

# Agrupar os dados por UF e somar os valores de V1028 correspondentes
soma_v1028_uf = pnad2021.groupby('UF')['V1028'].sum()

# Convertendo a série em um data frame e renomeando a coluna
pop_total = soma_v1028_uf.to_frame(name='população').reset_index().\
    rename(columns={'index': 'UF'})

# criar um dicionário de mapeamento com as chaves em formato de strings
uf_map = {
    '11': 'RO',
    '12': 'AC',
    '13': 'AM',
    '14': 'RR',
    '15': 'PA',
    '16': 'AP',
    '17': 'TO',
    '21': 'MA',
    '22': 'PI',
    '23': 'CE',
    '24': 'RN',
    '25': 'PB',
    '26': 'PE',
    '27': 'AL',
    '28': 'SE',
    '29': 'BA',
    '31': 'MG',
    '32': 'ES',
    '33': 'RJ',
    '35': 'SP',
    '41': 'PR',
    '42': 'SC',
    '43': 'RS',
    '50': 'MS',
    '51': 'MT',
    '52': 'GO',
    '53': 'DF'
}

# Aplicando o mapeamento à coluna UF
pop_total['UF'] = pop_total['UF'].map(uf_map)

## arredondando os números de população e também tirando o 0.
pop_total['população'] = pop_total['população'].round(2).apply(lambda x: x[1:] if str(x).startswith('0') else x)

## Printando a população total estimada  
print(pop_total)


    UF   população
0   RO   1811173.0
1   AC    894396.0
2   AM   4125630.0
3   RR    578207.0
4   PA   8766892.0
5   AP    874822.0
6   TO   1600098.0
7   MA   7131998.0
8   PI   3291002.0
9   CE   9256370.0
10  RN   3570927.0
11  PB   4044899.0
12  PE   9637284.0
13  AL   3363296.0
14  SE   2345379.0
15  BA  14983546.0
16  MG  21442564.0
17  ES   4120396.0
18  RJ  17494589.0
19  SP  46770241.0
20  PR  11612238.0
21  SC   7356905.0
22  RS  11462947.0
23  MS   2778362.0
24  MT   3526243.0
25  GO   7242075.0
26  DF   3105838.0


Agora farei uma estimativa amostral de domicilios por unidade federativa utilizando o peso amostral. O resultado final abaixo é a estimativa amostral de domicílios por unidade federativa.  

In [39]:
## filtrando para para o chefe do domicílio. Para descobrir a quantidade de domicílios
domic_uf = pnad2021.query('V2005 == "01"').groupby('UF')['V1028'].sum()

##transformadno em data frame 
dom_total = domic_uf.to_frame().reset_index().rename(columns={'index': 'UF', 0: 'domic_uf'})

# Aplicando o mapeamento à coluna UF
dom_total['UF'] = dom_total['UF'].map(uf_map)

## arredondando os números de população e também tirando o 0.
dom_total['V1028'] = dom_total['V1028'].round(2).apply(lambda x: x[1:] if str(x).startswith('0') else x)

## Printando domicilio total estimado 
print(dom_total)


    UF        V1028
0   RO    628164.21
1   AC    269118.41
2   AM   1102468.67
3   RR    166456.45
4   PA   2586373.98
5   AP    246154.85
6   TO    524667.75
7   MA   2082838.89
8   PI   1025685.34
9   CE   3016222.18
10  RN   1167102.77
11  PB   1316365.69
12  PE   3231419.25
13  AL   1075138.15
14  SE    796790.00
15  BA   5156899.65
16  MG   7527204.85
17  ES   1462988.08
18  RJ   6546788.99
19  SP  16342583.68
20  PR   4066076.55
21  SC   2580998.51
22  RS   4337210.43
23  MS    950550.40
24  MT   1179946.31
25  GO   2496430.74
26  DF   1017831.68
